In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Load traffic and station data
traffic = pd.read_csv('traffic.csv')
stations = pd.read_csv('stations.csv')

# Melt hourly traffic data
hour_columns = [f'hour_{i:02d}' for i in range(24)]
traffic_long = traffic.melt(
    id_vars=['station_key', 'traffic_direction_seq', 'cardinal_direction_seq',
             'classification_seq', 'year', 'month', 'day', 'day_of_week',
             'public_holiday', 'school_holiday'],
    value_vars=hour_columns,
    var_name='hour',
    value_name='traffic_count'
)

# Convert hour column from 'hour_00' to int
traffic_long['hour'] = traffic_long['hour'].str.extract('hour_(\d+)').astype(int)

# Create datetime for each row
traffic_long['datetime'] = pd.to_datetime(traffic_long[['year', 'month', 'day']]) + pd.to_timedelta(traffic_long['hour'], unit='h')

df = traffic_long.merge(stations, on='station_key', how='left')

# Create basic features
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Drop irrelevant text columns (optional)
drop_cols = ['name', 'full_name', 'intersection']
df = df.drop(columns=drop_cols, errors='ignore')

# Encode categorical variables
categorical_cols = ['mab_way_type', 'road_functional_hierarchy', 'road_on_type',
                    'lane_count', 'road_classification_type', 'rms_region',
                    'lga', 'suburb']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Define features and target
X = df.drop(columns=['traffic_count', 'datetime', 'station_key'])  # Adjust as needed
y = df['traffic_count']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse:.2f}")

